In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
from keras.models import  Model
import tensorflow as tf
import numpy as np
import cv2
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Masking, Conv2D, Lambda, Flatten, Activation, Input, Reshape, LSTM, Dense, add, concatenate,BatchNormalization
from keras.optimizers import Adam, SGD
from keras import backend as K
import os
import pickle
import sys


/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
model=load_model('first_try_maxout_3.h5',custom_objects={'tf':tf})
model2 = Model(inputs=model.input,outputs=model.get_layer('maxout4').output)

In [4]:
model2.save_weights("model2.hdf5")

In [6]:
def max_out(inputs, num_units, axis=None):
    shape = inputs.get_shape().as_list()
    if shape[0] is None:
        shape[0] = -1
    if axis is None:  # Assume that channel is the last dimension
        axis = -1
    num_channels = shape[axis]
    if num_channels % num_units:
        raise ValueError('number of features({}) is not a multiple of num_units({})'.format(num_channels, num_units))
    shape[axis] = num_units
    shape += [num_channels // num_units]
    outputs = tf.reduce_max(tf.reshape(inputs, shape), -1, keepdims=False)
    return outputs

In [7]:
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args

    y_pred = y_pred[:, 2:, :]
    

    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

In [27]:
input_data = Input(name='the_input', shape=(32,581,3), dtype='float32')
inner = Conv2D(96, (9,9), padding='valid',name='conv1')(input_data)
inner=Lambda(max_out,arguments={'num_units':48},name='maxout1')(inner)
inner = Conv2D(128, (9,9), padding='valid',name='conv2')(inner)
inner=Lambda(max_out,arguments={'num_units':64},name='maxout2')(inner)
inner = Conv2D(256, (9,9), padding='valid',name='conv3')(inner)
inner=Lambda(max_out,arguments={'num_units':128},name='maxout3')(inner)
inner = Conv2D(512, (8,8), padding='valid',name='conv4')(inner)
output=Lambda(max_out,arguments={'num_units':128},name='maxout4')(inner)


In [28]:
model3 = Model(inputs=input_data, outputs=output)

In [29]:
model3.load_weights("model2.hdf5",by_name=True)

In [31]:
output_maxout = Reshape((550,128))(model3.output)

lstm_1=LSTM(128,return_sequences=True,kernel_initializer='he_normal',name='lstm1')(output_maxout)

lstm_1b=LSTM(128,return_sequences=True,go_backwards=True,kernel_initializer='he_normal',name='lstm1b')(output_maxout)

lstm_merge=concatenate([lstm_1,lstm_1b])

    # lstm_2=LSTM(128,return_sequences=True,kernel_initializer='he_normal',name='lstm2')(lstm_merge)
    #
    # lstm_2b=LSTM(128,return_sequences=True,go_backwards=True,kernel_initializer='he_normal',name='lstm2b')(lstm_merge)

    # inner=Dense(73,kernel_initializer='he_normal',name='dense')(concatenate([lstm_2,lstm_2b]))
inner = Dense(73, kernel_initializer='he_normal', name='dense')(lstm_merge)

y_pred = Activation('softmax',name='softmax')(inner)

labels = Input(name='the_labels', shape=[20], dtype='float32')

input_length = Input(name='input_length', shape=[1], dtype='int64')

labels_length = Input(name='label_length', shape=[1], dtype='int64')

loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([y_pred, labels, input_length, labels_length])

model4 = Model(inputs=[model3.input, labels, input_length, labels_length], outputs=loss_out)

In [32]:
model4.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input (InputLayer)          (None, 32, 581, 3)   0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 24, 573, 96)  23424       the_input[0][0]                  
__________________________________________________________________________________________________
maxout1 (Lambda)                (None, 24, 573, 48)  0           conv1[0][0]                      
__________________________________________________________________________________________________
conv2 (Conv2D)                  (None, 16, 565, 128) 497792      maxout1[0][0]                    
__________________________________________________________________________________________________
maxout2 (L